In [1]:
# Cell 0
# If at all possible please test locally or on the private tbears server. The Testnet
# is becoming cluttered with many deployments of Balanced contracts.
# Note that running on the private tbears server will require the number of top P-Reps 
# be set to 4 in the staking contract or it will fail to deploy.

network = "yeouido"  # set this to one of mainnet, yeouido, euljiro, pagoda, or custom

connections = {
"mainnet": {"iconservice": "https://ctz.solidwallet.io",       "nid": 1},
"yeouido": {"iconservice": "https://bicon.net.solidwallet.io", "nid": 3},
"gangnam": {"iconservice": "https://gicon.net.solidwallet.io", "nid": 3},
"hannam": {"iconservice": "https://hannam.net.solidwallet.io", "nid": 3},
"euljiro": {"iconservice": "https://test-ctz.solidwallet.io",  "nid": 2},
"pagoda":  {"iconservice": "https://zicon.net.solidwallet.io", "nid":80},
"custom":  {"iconservice": "http://18.144.108.38:9000",        "nid": 3}}

env = connections[network]

In [2]:
# Cell 1
import sys
import time
sys.path.append("..")
from iconsdk.exception import JSONRPCException
from iconsdk.libs.in_memory_zip import gen_deploy_data_content
from iconsdk.icon_service import IconService
from iconsdk.providers.http_provider import HTTPProvider
from iconsdk.builder.transaction_builder import CallTransactionBuilder, TransactionBuilder, DeployTransactionBuilder
from iconsdk.builder.call_builder import CallBuilder
from iconsdk.signed_transaction import SignedTransaction
from iconsdk.wallet.wallet import KeyWallet
from iconsdk.utils.convert_type import convert_hex_str_to_int
from repeater import retry
from shutil import make_archive
import pickle as pkl
from datetime import datetime
from time import sleep
import json
import os


ICX = 1000000000000000000 # 10**18
GOVERNANCE_ADDRESS = "cx0000000000000000000000000000000000000000"
TEST_ORACLE = "cx61a36e5d10412e03c907a507d1e8c6c3856d9964"
MAIN_ORACLE = "cxe647e0af68a4661566f5e9861ad4ac854de808a2"
BALANCED_TEST = "hx3f01840a599da07b0f620eeae7aa9c574169a4be"

@retry(JSONRPCException, tries=10, delay=1, back_off=2)
def get_tx_result(_tx_hash):
    tx_result = icon_service.get_transaction_result(_tx_hash)
    return tx_result

In [3]:
# Cell 2

icon_service = IconService(HTTPProvider(env["iconservice"], 3))
NID = env["nid"]

In [4]:
# Cell 3

wallet = KeyWallet.load("../keystores/keystore_test1.json", "test1_Account")
# Balanced test wallet
with open("../keystores/balanced_test.pwd", "r") as f:
    key_data = f.read()
btest_wallet = KeyWallet.load("../keystores/balanced_test.json", key_data)
with open("../keystores/staking_test.pwd", "r") as f:
    key_data = f.read()
staking_wallet = KeyWallet.load("../keystores/staking_test.json", key_data)


In [5]:
print(wallet.get_address())
print(icon_service.get_balance(wallet.get_address()) / 10**18)

hxe7af5fcfd8dfc67530a01a0e403882687528dfcb
2137.702344229458


In [22]:
print(btest_wallet.get_address())
print(icon_service.get_balance(btest_wallet.get_address()) / 10**18)

hx3f01840a599da07b0f620eeae7aa9c574169a4be
767844.4277865621


In [6]:
user1 = KeyWallet.load("../keystores/user1.json","HelloWorld@1234")
# btest_wallet = KeyWallet.load("./balanced_test.json","HelloWorld@1234")

print(icon_service.get_balance(user1.get_address())/10**18)
print(user1.get_address())

# test2 = hx7a1824129a8fe803e45a3aae1c0e060399546187
private = "0a354424b20a7e3c55c43808d607bddfac85d033e63d7d093cb9f0a26c4ee022"
user2 = KeyWallet.load(bytes.fromhex(private))
print(icon_service.get_balance(user2.get_address())/10**18)
print(user2.get_address())

2236.3532934
hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db
2390.4148214
hx7a1824129a8fe803e45a3aae1c0e060399546187


In [7]:
# Cell 5a
# The following addresses are those deployed to the private tbears server.

contracts = {'loans': {'zip': 'core_contracts/loans.zip',
  'SCORE': 'cx3259f3ff9a51ca3bf170d4ff9104cf4af126ca1c'},
 'staking': {'zip': 'core_contracts/staking.zip',
  'SCORE': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a'},
 'dividends': {'zip': 'core_contracts/dividends.zip',
  'SCORE': 'cx1379084f45776301abda3849c6e374f460ee0155'},
 'reserve': {'zip': 'core_contracts/reserve.zip',
  'SCORE': 'cx71dda2221bf88faddc8f84b72ffd6db296e5609e'},
 'daofund': {'zip': 'core_contracts/daofund.zip',
  'SCORE': 'cxfd09787f23d23b945fa0c7eb55b5aa69425da1c8'},
 'rewards': {'zip': 'core_contracts/rewards.zip',
  'SCORE': 'cx893fccdd0881d8e2bd2625f711b38e06848ecb89'},
 'dex': {'zip': 'core_contracts/dex.zip',
  'SCORE': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62'},
 'governance': {'zip': 'core_contracts/governance.zip',
  'SCORE': 'cx483630769b61b76387d6ed90c911c16da546784f'},
 'oracle': {'zip': 'core_contracts/oracle.zip',
  'SCORE': 'cxed97bdb35a7ca1b3993e400e4dba9e11610338f7'},
 'sicx': {'zip': 'token_contracts/sicx.zip',
  'SCORE': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d'},
 'bnUSD': {'zip': 'token_contracts/bnUSD.zip',
  'SCORE': 'cxc48c9c81ceef04445c961c5cc8ff056d733dfe3a'},
 'bnXLM': {'zip': 'token_contracts/bnXLM.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'bnDOGE': {'zip': 'token_contracts/bnDOGE.zip',
  'SCORE': 'cx266bdc0c35828c8130cdf1cbaa3ad109f7694722'},
 'baln': {'zip': 'token_contracts/baln.zip',
  'SCORE': 'cx4d0768508a7ff550de4405f27aebfb8831565c19'},
 'bwt': {'zip': 'token_contracts/bwt.zip',
  'SCORE': 'cx663f9d59163846d9f6c6f7b586858c59aa8878a9'}}

In [10]:
contracts = {"loans": "cx3259f3ff9a51ca3bf170d4ff9104cf4af126ca1c", "staking": "cx9d829396d887f9292d8af488fab78ad24ab6b99a", "dividends": "cx203d9cd2a669be67177e997b8948ce2c35caffae", "reserve": "cx1754666c6779dc5e495a462144dd15e4a68fe776", "daofund": "cx430955c5a5e2a6e48c1f2aaa7258dc4c84222247", "rewards": "cx893fccdd0881d8e2bd2625f711b38e06848ecb89", "dex": "cx399dea56cf199b1c9e43bead0f6a284bdecfbf62", "governance": "cx483630769b61b76387d6ed90c911c16da546784f", "oracle": "cx61a36e5d10412e03c907a507d1e8c6c3856d9964", "sicx": "cxae6334850f13dfd8b50f8544d5acb126bb8ef82d", "bnUSD": "cxc48c9c81ceef04445c961c5cc8ff056d733dfe3a", "bnXLM": "cx4a1179d3ff8c692c4b20bfab2a2038ef79811002", "bnDOGE": "cxa40ff2a028d4def5ae12ae82a188ad4eb051ab1f", "baln": "cx36169736b39f59bf19e8950f6c8fa4bfa18b710a", "bwt": "cx5d886977b7d24b9f73a460c9ca2d43847997c285"}

In [8]:
# Cell 6
# Define deploy and send_tx functions

def send_icx(to, amount, wallet):
    transaction = TransactionBuilder()\
        .from_(wallet.get_address())\
        .to(to)\
        .value(amount)\
        .step_limit(1000000) \
        .nid(NID) \
        .nonce(2) \
        .version(3) \
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    return icon_service.send_transaction(signed_transaction)


def compress():
    """
    Compress all SCORE folders in the core_contracts and token_contracts folders.
    Make sure the oracle address is correct.
    """
    deploy = list(contracts.keys())[:]
    for directory in {"../../core_contracts", "../../token_contracts"}:
        with os.scandir(directory) as it:
            for file in it:
                archive_name = directory + "/" + file.name
                if file.is_dir() and file.name in deploy:
                    make_archive(archive_name, "zip", directory, file.name)
                    contracts[file.name]['zip'] = archive_name + '.zip'
    if network == "yeouido":
        contracts['oracle']['SCORE'] = TEST_ORACLE
    elif network == "mainnet":
        contracts['oracle']['SCORE'] = MAIN_ORACLE
                    

def deploy_SCORE(contract, params, wallet, update) -> str:
    """
    contract is of form {'zip': 'core_contracts/governance.zip', 'SCORE': 'cx1d81f93b3b8d8d2a6455681c46128868782ddd09'}
    params is a dicts
    wallet is a wallet file
    update is boolian
    """
    print(f'{contract["zip"]}')
    if update:
        dest = contract['SCORE']
    else:
        dest = GOVERNANCE_ADDRESS
    zip_file = contract['zip']
    step_limit = 3000000000
    deploy_transaction = DeployTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(dest)\
        .nid(NID)\
        .nonce(100)\
        .content_type("application/zip")\
        .content(gen_deploy_data_content(zip_file))\
        .params(params)\
        .build()

    signed_transaction = SignedTransaction(deploy_transaction, wallet, step_limit)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    print(f'Status: {res["status"]}')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        print(f'Failure: {res["failure"]}')
    print('')
    return res

def send_tx(dest, value, method, params, wallet, _print = True):
    """
    dest is the name of the destination contract.
    """
    if _print:
        print('------------------------------------------------------------------------------------------------------------------')
        print(f'Calling {method}, with parameters {params} on the {dest} contract.')
        print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)

    res = get_tx_result(tx_hash)
    if _print:
        print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        if _print:
            print(f'Failure: {res["failure"]}')
    return res

def deploy_all(wallet, staking_wallet):
    """
    Compress and Deploy all SCOREs.
    """
    compress()

    deploy = list(contracts.keys())[:]
    deploy.remove('oracle')
    deploy.remove('staking')
    deploy.remove('sicx')
    deploy.remove('governance')
    if network == "mainnet":
        if 'bnXLM' in deploy:
            deploy.remove('bnXLM')
        if 'bnDOGE' in deploy:
            deploy.remove('bnDOGE')

    results = {}
    res = deploy_SCORE(contracts['governance'], {}, wallet, 0)
    results[f'{contracts["governance"]}|deploy|{{}}'] = res
    governance = res.get('scoreAddress', '')
    contracts['governance']['SCORE'] = governance
    params = {'_governance': governance}
    for score in deploy:
        res = deploy_SCORE(contracts[score], params, wallet, 0)
        results[f'{contracts[score]}|deploy|{params}'] = res
        contracts[score]['SCORE'] = res.get('scoreAddress', '')

    res = deploy_SCORE(contracts['staking'], {}, staking_wallet, 0)
    results[f'{contracts["staking"]}|deploy|{{}}'] = res
    contracts['staking']['SCORE'] = res.get('scoreAddress', '')

    params = {'_admin': contracts['staking']['SCORE']}
    res = deploy_SCORE(contracts['sicx'], params, staking_wallet, 0)
    results[f'{contracts["sicx"]}|deploy|{params}'] = res
    contracts['sicx']['SCORE'] = res.get('scoreAddress', '')
    
    return results

    
def config_balanced(wallet, staking_wallet):
    """
    Configure all SCOREs before launch.
    """
    config = list(contracts.keys())[:]
    config.remove('governance')
    config.remove('bnDOGE')
    config.remove('bnXLM')
    addresses = {contract: contracts[contract]['SCORE'] for contract in config}
    txns = [{'contract': 'governance', 'value': 0, 'method': 'setAddresses', 'params': {'_addresses': addresses}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'setSicxAddress', 'params': {'_address': contracts['sicx']['SCORE']}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'configureBalanced', 'params': {}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results

def launch_balanced(wallet, staking_wallet):
    """
    Launch Balanced, turn on staking management, and set delegation for sICX on the Loans contract.
    """
    if network == "custom":
        preps = {
            "hx9eec61296a7010c867ce24c20e69588e2832bc52",  # ICX Station
            "hx000e0415037ae871184b2c7154e5924ef2bc075e"}  # iBriz-ICONOsphere
    elif network == "yeouido":
        preps = {
            "hx23823847f593ecb65c9e1ea81a789b02766280e8",  # ICX Station
            "hxe0cde6567eb6529fe31b0dc2f2697af84847f321",  # iBriz-ICONOsphere
            "hx83c0fc2bcac7ecb3928539e0256e29fc371b5078",  # Mousebelt
            "hx48b4636e84d8c491c88c18b65dceb7598c4600cc",  # Parrot 9
            "hxb4e90a285a79687ec148c29faabe6f71afa8a066"}  # ICONDAO
    elif network == "mainnet":
        preps = {
            "hxfba37e91ccc13ec1dab115811f73e429cde44d48",  # ICX Station
            "hx231a795d1c719b9edf35c46b9daa4e0b5a1e83aa",  # iBriz-ICONOsphere
            "hxbc9c73670c79e8f6f8060551a792c2cf29a8c491",  # Mousebelt
            "hx28c08b299995a88756af64374e13db2240bc3142"}  # Parrot 9
    else:
        return

    txns = [{'contract': 'governance', 'value': 0, 'method': 'launchBalanced', 'params': {}, 'wallet': wallet},
            {'contract': 'staking', 'value': 0, 'method': 'toggleStakingOn', 'params': {}, 'wallet': staking_wallet},
            {'contract': 'governance', 'value': 0, 'method': 'delegate', 'params': {'_delegations': [{'_address': prep, '_votes_in_per': str(100 * ICX // len(preps))} for prep in preps]}, 'wallet': wallet}]

    results = {}
    for tx in txns:
        res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], tx["wallet"])
        results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

    return results
   

def get_scores_json(contracts):
    """
    Prints out dictionary of SCORE addresses for use in testing UI.
    """
    scores = {}
    for score in contracts:
        scores[score] = contracts[score]['SCORE']
    return json.dumps(scores)

def call_tx(dest: str, method: str, params: dict = {}, _print = True):
    """
    dest is the name of the destination contract.
    """
    if _print:
        print('------------------------------------------------------------------------------------------------------------------')
        print(f'Reading {method}, with parameters {params} on the {dest} contract.')
        print('------------------------------------------------------------------------------------------------------------------')
    call = CallBuilder()\
        .from_(btest_wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .method(method)\
        .params(params)\
        .build()
    result = icon_service.call(call)
    if _print:
        print(result)
    return result

def fast_send_tx(dest, value, method, params, wallet, _print = False):
    """
    dest is the name of the destination contract.
    """
    if _print:
        print('------------------------------------------------------------------------------------------------------------------')
        print(f'Calling {method}, with parameters {params} on the {dest} contract.')
        print('------------------------------------------------------------------------------------------------------------------')
    transaction = CallTransactionBuilder()\
        .from_(wallet.get_address())\
        .to(contracts[dest]['SCORE'])\
        .value(value)\
        .step_limit(10000000)\
        .nid(NID)\
        .nonce(100)\
        .method(method)\
        .params(params)\
        .build()
    signed_transaction = SignedTransaction(transaction, wallet)
    tx_hash = icon_service.send_transaction(signed_transaction)
    
    res = get_tx_result(tx_hash)
    if _print:
        print(f'************************************************** Status: {res["status"]} **************************************************')
    if len(res["eventLogs"]) > 0:
        for item in res["eventLogs"]:
            print(f'{item} \n')
    if res['status'] == 0:
        if _print:
            print(f'Failure: {res["failure"]}')
    return res



In [ ]:
# Cell 7
# Deploy and configure Balanced. Print results if anything goes wrong.

if network == 'custom':
    confirm = 'Yes'
else:
    confirm = input(f'Deploying Balanced to {network}. Proceed (Yes/No)? ')
if confirm == 'Yes':
    results = {}
    deploy_all(btest_wallet, staking_wallet)
    print('------------------------------------------------------------------------------------------------------------------')
    print(contracts)
    print('----------Contracts for Testing UI--------------------------------------------------------------------------------')
    print(get_scores_json(contracts))

In [ ]:
# Cell 7a
# Configure Balanced

config_results = config_balanced(btest_wallet, staking_wallet)
print(config_results)

In [ ]:
# Cell 7b
# Launch Balanced
# We may want to make this a payable method and have the governance SCORE borrow bnUSD,
# start and name the sICXbnUSD market, and add it as a rewards DataSource.

launch_results = launch_balanced(btest_wallet, staking_wallet)
print(launch_results)

In [30]:

call = CallBuilder().from_(wallet.get_address())\
                    .to('cxd4534956f0b749be87b8128b5f728166c1417a00')\
                    .method('balanceOf')\
                    .params({'_owner': btest_wallet.get_address()}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

1000.0

In [28]:
# Create a new pool and generate assets for it

omm_amount = 100 * ICX
sICX_amount = 200 * ICX
icx_price = 2
sICX_contribute_amount = int(100 * ICX)
omm_contribute_amount = int(50 * ICX)

txns = [
#     {'contract': 'staking', 'value': sICX_amount, 'method': 'stakeICX', 'params': {}},
        {'contract': 'sicx', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': sICX_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
        {'contract': 'omm', 'value': 0, 'method': 'transfer', 'params': {'_to': contracts['dex']['SCORE'], '_value': omm_amount, '_data': json.dumps({"method": "_deposit"}).encode()}},
        {'contract': 'dex', 'value': 0, 'method': 'add', 'params': {'_baseToken': 'cxd4534956f0b749be87b8128b5f728166c1417a00', '_quoteToken': contracts['sicx']['SCORE'], '_baseValue': omm_contribute_amount, '_quoteValue': sICX_contribute_amount}}]

supply_liquidity_results = {}
for tx in txns:
    res = send_tx(tx["contract"], tx["value"], tx["method"], tx["params"], btest_wallet)
    supply_liquidity_results[f'{tx["contract"]}|{tx["method"]}|{tx["params"]}'] = res

------------------------------------------------------------------------------------------------------------------
Calling transfer, with parameters {'_to': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', '_value': 200000000000000000000, '_data': b'{"method": "_deposit"}'} on the sicx contract.
------------------------------------------------------------------------------------------------------------------


Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


************************************************** Status: 0 **************************************************
Failure: {'code': 1, 'message': 'Insufficient balance.'}
------------------------------------------------------------------------------------------------------------------
Calling transfer, with parameters {'_to': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', '_value': 100000000000000000000, '_data': b'{"method": "_deposit"}'} on the omm contract.
------------------------------------------------------------------------------------------------------------------


KeyError: 'omm'

In [33]:
# Cell 11
# transfer sicx to the wallet

params = {'_to': btest_wallet.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(1000 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("stakeICX")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Status: 1
{'scoreAddress': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a', 'indexed': ['UnstakeAmountTransfer(Address,int)', 'hxba1fabbda318438fcd93e92dde8936c649848a58', '0x7021ed30b927ce05'], 'data': []} 

{'scoreAddress': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx9d829396d887f9292d8af488fab78ad24ab6b99a', 'hxba1fabbda318438fcd93e92dde8936c649848a58', '0x7021ed30b927ce05'], 'data': []} 

{'scoreAddress': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a', 'indexed': ['FundTransfer(Address,int,str)', 'hxba1fabbda318438fcd93e92dde8936c649848a58', '0x7021ed30b927ce05'], 'data': [' 8079999999999987205 ICX sent to hxba1fabbda318438fcd93e92dde8936c649848a58.']} 

{'scoreAddress': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a', 'indexed': ['UnstakeAmountTransfer(Address,int)', 'hxba1fabbda318438fcd93e92dde8936c649848a58', '0x120c4ddc126540c1e'], 'data': []} 

{'scoreAddress': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a', 'indexed': ['ICXTransfer(

In [34]:
# Cell 14
# deposit sicx to the dex to create the pool

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 200 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Status: 1
{'scoreAddress': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', '0xad78ebc5ac6200000'], 'data': ['0x7b226d6574686f64223a20225f6465706f736974227d']} 

{'scoreAddress': 'cx893fccdd0881d8e2bd2625f711b38e06848ecb89', 'indexed': ['Report(int,str,int,int)', '0x19', 'OMM/sICX'], 'data': ['0x0', '0x0']} 

{'scoreAddress': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', 'indexed': ['Deposit(Address,Address,int)', 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'], 'data': ['0xad78ebc5ac6200000']} 



In [36]:
# Cell 16a
# deposit ommtoken to dex to create a pool

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 100 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to('cxd4534956f0b749be87b8128b5f728166c1417a00')\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Status: 1
{'scoreAddress': 'cx893fccdd0881d8e2bd2625f711b38e06848ecb89', 'indexed': ['Report(int,str,int,int)', '0x17', 'OMM/USDb'], 'data': ['0x0', '0x0']} 

{'scoreAddress': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', 'indexed': ['Deposit(Address,Address,int)', 'cxd4534956f0b749be87b8128b5f728166c1417a00', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'], 'data': ['0x56bc75e2d63100000']} 

{'scoreAddress': 'cxd4534956f0b749be87b8128b5f728166c1417a00', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', '0x56bc75e2d63100000'], 'data': ['0x7b226d6574686f64223a20225f6465706f736974227d']} 



In [57]:
# Cell 17a
# add base and quote token addresses for the paris that we want to create a pool of omm/sicx.

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken':'cxd4534956f0b749be87b8128b5f728166c1417a00' , '_quoteToken': contracts['sicx']['SCORE'], '_baseValue': 50 * ICX, '_quoteValue': 100 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Status: 1
{'scoreAddress': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', 'indexed': ['MarketAdded(int,Address,Address,int,int)', '0x8', 'cxd4534956f0b749be87b8128b5f728166c1417a00', 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d'], 'data': ['0x2b5e3af16b1880000', '0x56bc75e2d63100000']} 

{'scoreAddress': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', 'indexed': ['Add(int,Address,int,int,int)', '0x8', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', '0x3d54ecb1b916736b8'], 'data': ['0x2b5e3af16b1880000', '0x56bc75e2d63100000']} 

{'scoreAddress': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', 'indexed': ['TransferSingle(Address,Address,Address,int,int)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', 'cxf000000000000000000000000000000000000000', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'], 'data': ['0x8', '0x3d54ecb1b916736b8']} 

{'scoreAddress': 'cxd4534956f0b749be87b8128b5f728166c1417a00', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', 'hx3

In [25]:
# Cell 18a
# setting market name for ommsICX pair

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_id':8, '_name': 'OMM2/sICX'}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Status: 1


['sICX/ICX', 'sICX/bnUSD', 'BALN/bnUSD', 'BALN/sICX', 'OMM/IUSDC', 'OMM/sICX', 'OMM/USDb', 'OMM2/sICX', 'OMM2/USDS', 'OMM2/IUSDC']


In [26]:

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPriceByName')\
                    .params({'_name': 'OMM2/sICX'}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

2.0

In [ ]:
Omm/usds

In [9]:
# Cell 16a
# deposit ommtoken to dex to create a pool

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 100 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to('cxd4534956f0b749be87b8128b5f728166c1417a00')\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Status: 1
{'scoreAddress': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', 'indexed': ['Deposit(Address,Address,int)', 'cxd4534956f0b749be87b8128b5f728166c1417a00', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'], 'data': ['0x56bc75e2d63100000']} 

{'scoreAddress': 'cxd4534956f0b749be87b8128b5f728166c1417a00', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', '0x56bc75e2d63100000'], 'data': ['0x7b226d6574686f64223a20225f6465706f736974227d']} 



In [12]:
# Cell 16a
# deposit usds to dex to create a pool

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 100 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to('cxaa068556df80f9917ef146e889f0b2c4b13ab634')\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Status: 0
Failure: {'code': 32, 'message': 'Stably USD: Out of balance'}


In [13]:
# Cell 17a
# add base and quote token addresses for the paris that we want to create a pool of omm/sicx.

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken':'cxd4534956f0b749be87b8128b5f728166c1417a00' , '_quoteToken': 'cxaa068556df80f9917ef146e889f0b2c4b13ab634', '_baseValue': 50 * ICX, '_quoteValue': 100 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Status: 0
Failure: {'code': 32, 'message': 'Balanced DEX: insufficient quote asset funds deposited.'}


In [23]:
# Cell 18a
# setting market name for ommsICX pair

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_id':9, '_name': 'OMM2/USDS'}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Status: 1


['sICX/ICX', 'sICX/bnUSD', 'BALN/bnUSD', 'BALN/sICX', 'OMM/IUSDC', 'OMM/sICX', 'OMM/USDb', 'OMM2/sICX', 'OMM2/USDS', 'OMM2/IUSDC']


In [24]:

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPriceByName')\
                    .params({'_name': 'OMM2/USDS'}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

2.0

omm/iusdc

In [74]:
# Cell 16a
# deposit ommtoken to dex to create a pool

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 100 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to('cxd4534956f0b749be87b8128b5f728166c1417a00')\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Status: 1
{'scoreAddress': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', 'indexed': ['Deposit(Address,Address,int)', 'cxd4534956f0b749be87b8128b5f728166c1417a00', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be'], 'data': ['0x56bc75e2d63100000']} 

{'scoreAddress': 'cxd4534956f0b749be87b8128b5f728166c1417a00', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx3f01840a599da07b0f620eeae7aa9c574169a4be', 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', '0x56bc75e2d63100000'], 'data': ['0x7b226d6574686f64223a20225f6465706f736974227d']} 



In [17]:
# Cell 16a
# deposit iusdc to dex to create a pool

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 200 * 10**6, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to('cx65f639254090820361da483df233f6d0e69af9b7')\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Status: 1
{'scoreAddress': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', 'indexed': ['Snapshot(int)', '0x1e'], 'data': []} 

{'scoreAddress': 'cx36169736b39f59bf19e8950f6c8fa4bfa18b710a', 'indexed': ['Mint(Address,int,bytes)', 'cx893fccdd0881d8e2bd2625f711b38e06848ecb89'], 'data': ['0x152d02c7e14af6800000', '0x4e6f6e65']} 

{'scoreAddress': 'cx36169736b39f59bf19e8950f6c8fa4bfa18b710a', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hx0000000000000000000000000000000000000000', 'cx893fccdd0881d8e2bd2625f711b38e06848ecb89', '0x152d02c7e14af6800000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx36169736b39f59bf19e8950f6c8fa4bfa18b710a', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx893fccdd0881d8e2bd2625f711b38e06848ecb89', 'cx5d886977b7d24b9f73a460c9ca2d43847997c285', '0x43c33c1937564800000'], 'data': ['0x4e6f6e65']} 

{'scoreAddress': 'cx36169736b39f59bf19e8950f6c8fa4bfa18b710a', 'indexed': ['Transfer(Address,Address,int,bytes)', 'cx893fccdd0881d8e2bd2625f711b38e06848ecb89

In [21]:
# Cell 17a

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken':'cxd4534956f0b749be87b8128b5f728166c1417a00' , '_quoteToken': 'cx65f639254090820361da483df233f6d0e69af9b7', '_baseValue': 50 * ICX, '_quoteValue': 100 * 10**6}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Status: 0
Failure: {'code': 32, 'message': 'Balanced DEX Rewards distribution in progress, please try again shortly'}


In [ ]:

to_token = 'cx65f639254090820361da483df233f6d0e69af9b7'
params_data = "{\"method\": \"_swap\", \"params\": {\"toToken\":\"" + str(to_token) + "\", \"maxSlippage\":250}}"
data = params_data.encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': (int) (10 * ICX), '_data': data}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to('cxd4534956f0b749be87b8128b5f728166c1417a00')\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

In [39]:
# Cell 18a
# setting market name for ommsICX pair

transaction = CallTransactionBuilder()\
    .from_(btest_wallet.get_address())\
    .to(contracts['governance']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("setMarketName")\
    .params({'_id':10, '_name': 'OMM2/IUSDC'}) \
    .build()
signed_transaction = SignedTransaction(transaction, btest_wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')
print('\n')

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getNamedPools')\
                    .build()
result = icon_service.call(call)
print(result)

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Status: 1


['sICX/ICX', 'sICX/bnUSD', 'BALN/bnUSD', 'BALN/sICX', 'OMM/IUSDC', 'OMM/sICX', 'OMM/USDb', 'OMM2/sICX', 'OMM2/USDS', 'OMM2/IUSDC']


In [28]:

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPriceByName')\
                    .params({'_name': 'OMM2/IUSDC'}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**6

2.0

In [50]:
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolStats')\
                    .params({'_id': 10}) \
                    .build()
result = icon_service.call(call)
print(result)


{'base': '0x56bc75e2d63100000', 'quote': '0xbebc200', 'base_token': 'cxd4534956f0b749be87b8128b5f728166c1417a00', 'quote_token': 'cx65f639254090820361da483df233f6d0e69af9b7', 'total_supply': '0x809f39b3c1fc', 'price': '0x1e8480', 'name': 'OMM2/IUSDC', 'base_decimals': '0x12', 'quote_decimals': '0x6', 'min_quote': '0x0'}


In [32]:
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolStats')\
                    .params({'_id': 10}) \
                    .build()
result = icon_service.call(call)
print(result)


{'base': '0x56bc75e2d63100000', 'quote': '0xbebc200', 'base_token': 'cxd4534956f0b749be87b8128b5f728166c1417a00', 'quote_token': 'cx65f639254090820361da483df233f6d0e69af9b7', 'total_supply': '0x809f39b3c1fc', 'price': '0x1e8480', 'name': 'OMM2/IUSDC', 'base_decimals': '0x12', 'quote_decimals': '0x6', 'min_quote': '0x0'}


Add to pool from different wallet

In [8]:
# Cell 11
# transfer sicx to the wallet

params = {'_to': user1.get_address()}
transaction = CallTransactionBuilder()\
    .from_(wallet.get_address())\
    .to(contracts['staking']['SCORE'])\
    .value(1000 * ICX)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("stakeICX")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, wallet)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Status: 1
{'scoreAddress': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a', 'indexed': ['UnstakeAmountTransfer(Address,int)', 'hxd36e47b1c264ffc6a3a7bea810273ce7fb121836', '0xe1b001966bca364ff'], 'data': []} 

{'scoreAddress': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a', 'indexed': ['ICXTransfer(Address,Address,int)', 'cx9d829396d887f9292d8af488fab78ad24ab6b99a', 'hxd36e47b1c264ffc6a3a7bea810273ce7fb121836', '0xe1b001966bca364ff'], 'data': []} 

{'scoreAddress': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a', 'indexed': ['FundTransfer(Address,int,str)', 'hxd36e47b1c264ffc6a3a7bea810273ce7fb121836', '0xe1b001966bca364ff'], 'data': [' 260199999999999960319 ICX sent to hxd36e47b1c264ffc6a3a7bea810273ce7fb121836.']} 

{'scoreAddress': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a', 'indexed': ['UnstakeAmountTransfer(Address,int)', 'hxd36e47b1c264ffc6a3a7bea810273ce7fb121836', '0x136d858738ac7d5d96'], 'data': []} 

{'scoreAddress': 'cx9d829396d887f9292d8af488fab78ad24ab6b99a', 'indexed': ['ICXTra

In [9]:
# Cell 14
# deposit sicx to the dex to create the pool

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 200 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(user1.get_address())\
    .to(contracts['sicx']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, user1)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Status: 1
{'scoreAddress': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', '0xad78ebc5ac6200000'], 'data': ['0x7b226d6574686f64223a20225f6465706f736974227d']} 

{'scoreAddress': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', 'indexed': ['Deposit(Address,Address,int)', 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'], 'data': ['0xad78ebc5ac6200000']} 



In [11]:
# Cell 16a
# deposit ommtoken to dex to create a pool

data = "{\"method\": \"_deposit\"}".encode("utf-8")
params = {'_to': contracts['dex']['SCORE'], '_value': 100 * ICX, '_data': data}
transaction = CallTransactionBuilder()\
    .from_(user1.get_address())\
    .to('cxd4534956f0b749be87b8128b5f728166c1417a00')\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("transfer")\
    .params(params)\
    .build()
signed_transaction = SignedTransaction(transaction, user1)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Status: 1
{'scoreAddress': 'cx893fccdd0881d8e2bd2625f711b38e06848ecb89', 'indexed': ['Report(int,str,int,int)', '0x1e', 'Loans'], 'data': ['0x1a074d1a86f042f0c9', '0x65e523845d9a6d254']} 

{'scoreAddress': 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', 'indexed': ['Deposit(Address,Address,int)', 'cxd4534956f0b749be87b8128b5f728166c1417a00', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db'], 'data': ['0x56bc75e2d63100000']} 

{'scoreAddress': 'cxd4534956f0b749be87b8128b5f728166c1417a00', 'indexed': ['Transfer(Address,Address,int,bytes)', 'hxe62d11fa19a0e8575ad92f06bc8fd42edbfe27db', 'cx399dea56cf199b1c9e43bead0f6a284bdecfbf62', '0x56bc75e2d63100000'], 'data': ['0x7b226d6574686f64223a20225f6465706f736974227d']} 



In [62]:
# Cell 17a
# add base and quote token addresses for the paris that we want to create a pool of omm/sicx.

transaction = CallTransactionBuilder()\
    .from_(user1.get_address())\
    .to(contracts['dex']['SCORE'])\
    .value(0)\
    .step_limit(10000000)\
    .nid(NID)\
    .nonce(100)\
    .method("add")\
    .params({'_baseToken':'cxd4534956f0b749be87b8128b5f728166c1417a00' , '_quoteToken': contracts['sicx']['SCORE'], '_baseValue': 100 * ICX, '_quoteValue': 200 * ICX}) \
    .build()
signed_transaction = SignedTransaction(transaction, user1)
tx_hash = icon_service.send_transaction(signed_transaction)
tx_hash

res = get_tx_result(tx_hash)
print(f'Status: {res["status"]}')
if len(res["eventLogs"]) > 0:
    for item in res["eventLogs"]:
        print(f'{item} \n')
if res['status'] == 0:
    print(f'Failure: {res["failure"]}')

Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}
Raised JSONRPCException while returning the custom response. Error message: {'code': -32602, 'message': 'Pending transaction'}


Status: 0
Failure: {'code': 32, 'message': 'Balanced DEX Rewards distribution in progress, please try again shortly'}


In [ ]:

call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPriceByName')\
                    .params({'_name': 'OMM2/sICX'}) \
                    .build()
result = icon_service.call(call)
int(result, 0) / 10**18

In [12]:
call = CallBuilder().from_(wallet.get_address())\
                    .to(contracts['dex']['SCORE'])\
                    .method('getPoolStats')\
                    .params({'_id': 8}) \
                    .build()
result = icon_service.call(call)
print(result)


{'base': '0x2b5e3af16b1880000', 'quote': '0x56bc75e2d63100000', 'base_token': 'cxd4534956f0b749be87b8128b5f728166c1417a00', 'quote_token': 'cxae6334850f13dfd8b50f8544d5acb126bb8ef82d', 'total_supply': '0x3d54ecb1b916736b8', 'price': '0x1bc16d674ec80000', 'name': 'OMM2/sICX', 'base_decimals': '0x12', 'quote_decimals': '0x12', 'min_quote': '0x88c3ddbf4b4476f7'}
